In [1]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd

head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}


In [ ]:
df = pd.read_csv("game_contents.csv")

In [ ]:
for game_id in df['id']:
    link = f'https://steamcommunity.com/app/{game_id}/reviews/'
    req = requests.get(link, headers=head)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    

In [12]:
link = f'https://steamcommunity.com/app/1593500/reviews/?p=1&browsefilter=toprated'
req = requests.get(link, headers=head)
soup = BeautifulSoup(req.text, 'html.parser')

In [13]:
len(soup.find_all("div", {"class":"apphub_Card modalContentLink interactable"}))

10